<a href="https://colab.research.google.com/github/VavRe/nlp-ut/blob/main/CA2/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir ~/.kaggle/
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ ;
!chmod 600 ~/.kaggle/kaggle.json ; 
!kaggle datasets download -d ankurzing/sentiment-analysis-for-financial-news ;
!unzip /content/sentiment-analysis-for-financial-news.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle/’: File exists
sentiment-analysis-for-financial-news.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/sentiment-analysis-for-financial-news.zip
replace FinancialPhraseBank/License.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = pd.read_csv("./all-data.csv", encoding='iso-8859-1', header=None)
dataset.columns = ["sentiment","phrase"]

In [4]:
dataset["sentiment"].value_counts()

neutral     2879
positive    1363
negative     604
Name: sentiment, dtype: int64

In [5]:


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# def preprocess(text):
#     tokens = word_tokenize(text.lower())
#     filtered_tokens = [token for token in tokens if token not in stop_words]
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
#     preprocessed_text = ' '.join(lemmatized_tokens)
#     return preprocessed_text

In [6]:
import string
def preprocess_text(text):
    # tokenize the text
    tokens = word_tokenize(text)
    
    # initialize a list to store the preprocessed tokens
    preprocessed_tokens = []
    
    # iterate over the tokens
    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        # check if the token is a stop word or a punctuation
        if token.lower() not in stop_words and token not in string.punctuation:
            # lemmatize the token
            lemma = lemmatizer.lemmatize(token.lower())
            
            # check if the token is negative
            if i < len(tokens) - 1 and tokens[i+1] in ['not', "n't"]:
                preprocessed_tokens.append('NOT_' + lemma)
                i += 1
            elif lemma in ['negative', 'bad', 'poor', 'terrible', 'horrible']:
                preprocessed_tokens.append('NOT_' + lemma)
            else:
                preprocessed_tokens.append(lemma)
        
        i += 1
    
    return preprocessed_tokens

In [7]:
encoder = LabelEncoder()
dataset["label"] = encoder.fit_transform(dataset["sentiment"])
dataset

,sentiment,phrase,label
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,1
2,negative,The international electronic industry company ...,0
3,positive,With the new production plant the company woul...,2
4,positive,According to the company 's updated strategy f...,2
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,0
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,0
4844,negative,Net sales of the Paper segment decreased to EU...,0


In [8]:
dataset["phrase"] = dataset["phrase"].apply(preprocess_text)

In [9]:
dataset["phrase"]

0       [according, gran, company, plan, move, product...
1       [technopolis, plan, develop, stage, area, le, ...
2       [international, electronic, industry, company,...
3       [new, production, plant, company, would, incre...
4       [according, company, 's, updated, strategy, ye...
                              ...                        
4841    [london, marketwatch, --, share, price, ended,...
4842    [rinkuskiai, 's, beer, sale, fell, 6.5, per, c...
4843    [operating, profit, fell, eur, 35.4, mn, eur, ...
4844    [net, sale, paper, segment, decreased, eur, 22...
4845    [sale, finland, decreased, 10.5, january, sale...
Name: phrase, Length: 4846, dtype: object

In [10]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip -d glove.6B


--2023-05-06 14:37:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-06 14:37:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-06 14:37:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

In [11]:

embeddings_index = {}
with open('glove.6B/glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embeddings = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embeddings


In [12]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(dataset['phrase'])

phrases = tokenizer.texts_to_sequences(dataset['phrase'])
labels = dataset['label']

from keras.utils import pad_sequences

maxlen = 100
X = pad_sequences(phrases, padding='post', maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector



from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, stratify=labels)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.59      0.96      0.73       288
           2       0.24      0.03      0.05       136

    accuracy                           0.58       485
   macro avg       0.28      0.33      0.26       485
weighted avg       0.42      0.58      0.45       485



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
phrases[0]

[57, 3251, 3, 119, 519, 53, 89, 2555, 3, 631]

In [13]:
!pip install zeugma

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset["phrase"].to_list(), dataset["label"].to_list(), test_size=0.1, stratify=dataset["label"].to_list(), random_state=42)


In [15]:
from sklearn.linear_model import LogisticRegression
from zeugma.embeddings import EmbeddingTransformer
from sklearn.metrics import classification_report

glove = EmbeddingTransformer('glove')
X_train = glove.transform(X_train)

model = LogisticRegression()
model.fit(X_train, y_train)

X_test = glove.transform(X_test)
y_pred = model.predict(X_test)

target_names = ['Negative', 'Neutral', 'Positive']

print(classification_report(y_test, y_pred, target_names=target_names))


UnboundLocalError: ignored